In [ ]:
import findspark
findspark.init('/spark-3.5.1-bin-hadoop3')
from pyspark import *
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType, TimestampType, LongType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType, DecimalType
from pyspark.sql.functions import regexp_extract, split, from_unixtime, col, avg, min, max, desc
from pyspark.sql.functions import grouping, explode, array_contains
from pyspark.sql.functions import mean, stddev, skewness, kurtosis
import matplotlib.pyplot as plt
import pandas as pd
import random


spark = SparkSession.builder.appName("analytics").getOrCreate()

# Regresión Lineal

## Entrenamiento

### Generación de Datos

Primero generamos un conjunto de datos aleatorios que usaria Spark para entrenar el arbol de decisión

Características del conjunto de datos:

* Edad (entero): La edad del empleado.
* Educación (entero): Nivel de educación codificado como 0 (secundaria), 1 (pregrado), 2 (postgrado).
* Ingresos (flotante): Ingresos anuales del empleado en miles de dólares.

In [ ]:
# Generar datos
data = [
    Row(edad=random.randint(20, 60),
        educacion=random.choice([0, 1, 2]),
        ingresos=(random.randint(20, 60) * 1000) + (random.choice([0, 1, 2]) * 10000))
    for _ in range(300)
]

df = spark.createDataFrame(data)

# Guardar los datos en CSV
df.write.csv("datos_empleados.csv", header=True, mode="overwrite")

df.show(5)

## Preparar los datos


In [ ]:
from pyspark.ml.feature import VectorAssembler

# Assembler para combinar las características en una sola columna de vectores
assembler = VectorAssembler(
    inputCols=["edad", "educacion"],
    outputCol="features"
)
df_final = assembler.transform(df)
df_final.select("features", "ingresos").show(5)


### Entrenar el modelo de regresión lineal


In [ ]:
from pyspark.ml.regression import LinearRegression

# Crear y entrenar el modelo de regresión lineal
lr = LinearRegression(featuresCol="features", labelCol="ingresos")
model = lr.fit(df_final)

# Mostrar los coeficientes y el intercepto del modelo
print("Coeficientes: " + str(model.coefficients))
print("Intercepto: " + str(model.intercept))

## Evaluar el modelo


Para evaluar la efectividad de un modelo de regresión lineal, se utilizan varias métricas estadísticas, siendo el Error Cuadrático Medio (MSE, por sus siglas en inglés) una de las más comunes. El MSE mide el promedio de los cuadrados de los errores, es decir, la diferencia cuadrada entre los valores observados y los valores predichos por el modelo. Un MSE menor indica un mejor ajuste del modelo a los datos, ya que sugiere que las predicciones están más cercanas a los valores reales. Esta métrica es especialmente útil para comparar diferentes modelos o configuraciones del mismo modelo, ya que proporciona un indicador claro de mejora o deterioro en el rendimiento del modelo.

Otra métrica crucial es el Coeficiente de Determinación, conocido como `R^2`, que proporciona una medida de cuán bien los resultados observados son replicados por el modelo, basado en la proporción de la varianza total de los datos explicada por el modelo. El `R^2` es un número entre 0 y 1, donde un valor más alto indica que el modelo explica una mayor proporción de la variación en los datos, relacionándose esto con un mejor ajuste del modelo. Un `R^2` de 1 indica que el modelo ajusta perfectamente los datos con respecto a la variable dependiente. Sin embargo, un valor alto de `R^2` no siempre es sinónimo de un modelo predictivo efectivo, especialmente en casos donde la linealidad de los datos es cuestionable o cuando el modelo está sobreajustado.

In [ ]:
# Evaluar el modelo en el conjunto de entrenamiento
evaluation_summary = model.evaluate(df_final)

print("Error cuadrático medio (MSE):", evaluation_summary.meanSquaredError)
print("Coeficiente de determinación (R^2):", evaluation_summary.r2)
